In [5]:
from transformers import MarianMTModel, MarianTokenizer

# Function to translate text using MarianMT
def translate_text_with_marianmt(text, target_language="fr"):
    model_name = f'Helsinki-NLP/opus-mt-en-{target_language}'
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Translate the input text to the target language
    translated_text = model.generate(**tokenizer(text, return_tensors="pt"), max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(translated_text[0], skip_special_tokens=True)

# Define the input text
input_text = '''Art 8 • Vie privée et familiale • Refus des juridictions internes d’examiner 
l’action du requérant, affirmant être le père biologique d’un enfant, visant à 
contester la paternité légalement établie en vue de faire établir la sienne, en 
l’application des règles de computation du délai de forclusion de cinq ans 
combinées avec l’obligation d’attraire en la cause l’enfant • Requérant n’ayant 
pas agi dès la connaissance de sa paternité alors qu’il disposait d’un délai 
suffisant de plus de trois ans pour engager une action • Requérant ayant tardé à 
mettre dans la cause l’enfant sans justifier avoir pu ignorer l’existence de cette 
règle constante en droit interne • Conclusions des juridictions internes ni 
arbitraires ni déraisonnables • Refus fondé sur un lien de filiation déjà établi pour 
l’enfant et au regard de l’intérêt supérieur de ce dernier • Décisions judiciaires 
n’ayant pas abouti en pratique à priver le requérant de tout lien avec l’enfant • 
Juste équilibre ménagé entre les différents intérêts en présence'''

# Translate the text to the target language (e.g., "fr" for French)
translated_text = translate_text_with_marianmt(input_text, target_language="fr")

# Save the translated text to a file
with open("translated_text.txt", "w", encoding="utf-8") as file:
    file.write(translated_text)


In [5]:
# Input and output file paths
input_pdf_path = "uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"
output_pdf_path = "translated_outputBERT.pdf"

# Translate and create the new PDF
create_translated_pdf(input_pdf_path, output_pdf_path)

print(f"Translation complete. Translated PDF saved at: {output_pdf_path}")


Token indices sequence length is longer than the specified maximum sequence length for this model (17594 > 512). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [7]:
from transformers import MarianMTModel, MarianTokenizer

def translate_text_with_marianmt(text, target_language="en"):
    model_name = f'Helsinki-NLP/opus-mt-{target_language}-en'  # Replace "en" with the desired target language code
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)

    # Translate the input text to the target language
    translation_ids = model.generate(**inputs, num_beams=4, max_length=512, early_stopping=True)
    translated_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)

    return translated_text

# Example usage:
input_text = """Art 8 • Vie privée et familiale • Refus des juridictions internes d’examiner 
l’action du requérant, affirmant être le père biologique d’un enfant, visant à 
contester la paternité légalement établie en vue de faire établir la sienne, en 
l’application des règles de computation du délai de forclusion de cinq ans 
combinées avec l’obligation d’attraire en la cause l’enfant • Requérant n’ayant 
pas agi dès la connaissance de sa paternité alors qu’il disposait d’un délai 
suffisant de plus de trois ans pour engager une action • Requérant ayant tardé à 
mettre dans la cause l’enfant sans justifier avoir pu ignorer l’existence de cette 
règle constante en droit interne • Conclusions des juridictions internes ni 
arbitraires ni déraisonnables • Refus fondé sur un lien de filiation déjà établi pour 
l’enfant et au regard de l’intérêt supérieur de ce dernier • Décisions judiciaires 
n’ayant pas abouti en pratique à priver le requérant de tout lien avec l’enfant • 
Juste équilibre ménagé entre les différents intérêts en présence"""
target_language = "fr"  # Replace with the target language code, e.g., "fr" for French

translated_text = translate_text_with_marianmt(input_text, target_language)
print("Translated text:", translated_text)


Translated text: Art. 8 • Private and family life • Refusal of domestic courts to examine the applicant's action, claiming to be the biological father of a child, to challenge the legally established paternity with a view to establishing his/her own, in application of the rules for calculating the five-year period of foreclosure combined with the obligation of attrition in the case of the child • Complainant who did not act as soon as he/she knew his/her fatherhood when he/she had sufficient time to initiate an action • Complainant who delayed bringing the child into the case without having been able to ignore the existence of this constant rule in domestic law • Findings of internal courts neither arbitrary nor unreasonable • Refusal based on a filiation link already established for the child and in view of the best interests of the child • Judicial decisions which did not in practice lead to depriving the applicant of any connection with the child • Fair balance between the various i

In [10]:
from transformers import MarianMTModel, MarianTokenizer
from PyPDF2 import PdfReader

# Initialize the Marian NMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-fr-en"  # Replace with the model of your choice
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_text(text, target_language="en"):
    # Split the text into segments of 1024 tokens or less
    max_segment_length = 1024
    text_segments = [text[i:i+max_segment_length] for i in range(0, len(text), max_segment_length)]
    
    # Translate each segment and join the results
    translated_segments = []
    for segment in text_segments:
        inputs = tokenizer(segment, return_tensors="pt", max_length=max_segment_length, truncation=True, padding=True)
        translated = model.generate(**inputs, max_length=max_segment_length, num_beams=4, early_stopping=True, 
                                    length_penalty=2.0, no_repeat_ngram_size=3, num_return_sequences=1)
        translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        translated_segments.append(translated_text[0])
    
    return " ".join(translated_segments)

def translate_pdf(input_pdf_path, output_txt_path, target_language="en"):
    # Open the original PDF for reading
    pdf_reader = PdfReader(input_pdf_path)

    # Create a text file for writing
    with open(output_txt_path, "w", encoding="utf-8") as output_txt:
        # Iterate through pages in the original PDF
        for page in pdf_reader.pages:
            text = page.extract_text()
            translated_text = translate_text(text, target_language)
            output_txt.write(translated_text)

# # Example usage
# input_pdf_path = "input.pdf"  # Replace with the path to your input PDF file
# output_txt_path = "translated_output.txt"  # Replace with the desired output text file path
# target_language = "en"  # Replace with the target language code, e.g., "fr" for French

# translate_pdf(input_pdf_path, output_txt_path, target_language)


In [11]:
# Example usage
input_pdf_path = "./uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"  # Replace with the path to your input PDF file
output_pdf_path = "frMNMTtranslated_output.txt"  # Replace with the desired output PDF file path
target_language = ["en"]  # Replace with the target language code, e.g., "fr" for French

translate_pdf(input_pdf_path, output_pdf_path, target_language)

In [13]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

def create_pdf_from_text(input_txt_path, output_pdf_path):
    doc = SimpleDocTemplate(output_pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()

    translated_text = ""
    with open(input_txt_path, "r", encoding="utf-8") as input_txt:
        translated_text = input_txt.read()

    story = []
    paragraph = Paragraph(translated_text, styles["Normal"])
    story.append(paragraph)

    doc.build(story)

# Example usage
input_txt_path = "frMNMTtranslated_output.txt"  # Replace with the path to your translated text file
output_pdf_path = "frmmmttranslated_output.pdf"  # Replace with the desired output PDF file path

create_pdf_from_text(input_txt_path, output_pdf_path)
